(1) 모듈 로딩, 데이터 준비

In [1]:
# 모듈 로딩
from sklearn.datasets import fetch_openml
import numpy as np

In [2]:
db_name = 'Fashion-MNIST'
fashion_data = fetch_openml(name=db_name, parser='auto', as_frame=False)

In [3]:
fashion_data.__dir__()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [4]:
fashion_feature = fashion_data.data
fashion_target = fashion_data.target

In [5]:
type(fashion_feature), type(fashion_target)

(numpy.ndarray, numpy.ndarray)

In [6]:
fashion_feature.shape, fashion_target.shape

((70000, 784), (70000,))

In [7]:
fashion_feature.dtype, fashion_target.dtype

(dtype('int64'), dtype('O'))

In [8]:
unique_values = np.unique(fashion_feature)

In [9]:
unique_values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
# 고유한 값들과 각 값의 등장 횟수 얻기
unique_values, counts = np.unique(fashion_target, return_counts=True)

In [11]:
# target 고유값
unique_values

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

(2) 데이터 전처리

In [12]:
fashion_feature = fashion_feature / 255.    # 스케일링

In [13]:
fashion_feature[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
ohEncoder = OneHotEncoder(sparse_output=False)   # sparse_output=False 해줘야 transform 반환값이 ndarray (아니면 scipy.sparse._csr.csr_matrix)
ohEncoder.fit(fashion_target.reshape(-1, 1))    # 2D를 넘겨야 함 (기본적으로 여러 개의 컬럼을 기대함)
ohEncoder.categories_

[array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)]

In [16]:
norm_target = ohEncoder.transform(fashion_target.reshape(-1, 1))    # One-Hot-Encoding 적용한 y값

In [17]:
type(norm_target), norm_target.shape

(numpy.ndarray, (70000, 10))

In [18]:
# 훈련용, 테스트용 데이터 분리
x_train_arr = fashion_feature[:60000]
x_test_arr = fashion_feature[60000:]

y_train_arr = norm_target[:60000]
y_test_arr = norm_target[60000:]

In [19]:
x_train_arr.shape, y_train_arr.shape

((60000, 784), (60000, 10))

In [20]:
x_train_arr.dtype, y_train_arr.dtype

(dtype('float64'), dtype('float64'))

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [22]:
torch.manual_seed(1)

In [23]:
x_train = torch.FloatTensor(x_train_arr)
x_test = torch.FloatTensor(x_test_arr)

y_train = torch.FloatTensor(y_train_arr)
y_test = torch.FloatTensor(y_test_arr)

In [24]:
type(x_train.shape[0])

int

In [25]:
# 학습 인스턴스 생성
in_, out_ = x_train.shape[1], y_train.shape[1]
model = nn.Linear(in_, out_)
model

Linear(in_features=784, out_features=10, bias=True)

In [26]:
# 최적화 인스턴스 생성
optimizer = optim.Adam(model.parameters(), lr=1e-1)

In [27]:
# 학습 횟수
EPOCHS = 100

In [28]:
loss_list, accuracy_list = [], []

for epoch in range(EPOCHS):
    # 학습
    y_pred = model(x_train)         # y_pred : torch.Size([60000, 10])
    
    # 오차(손실) 계산
    loss = F.cross_entropy(y_pred, y_train)     # y_train : float 여야 함
    loss_list.append(loss)          # loss : torch.Tensor

    # W, b 업데이트
    optimizer.zero_grad()           # 기울기 초기화
    loss.backward()                 # 오차 역전파 계산
    optimizer.step()                # 새로운 값으로 W, b 업데이트

In [29]:
loss_list

[tensor(2.4191, grad_fn=<DivBackward1>),
 tensor(9.7125, grad_fn=<DivBackward1>),
 tensor(11.2556, grad_fn=<DivBackward1>),
 tensor(7.2877, grad_fn=<DivBackward1>),
 tensor(5.3443, grad_fn=<DivBackward1>),
 tensor(9.1352, grad_fn=<DivBackward1>),
 tensor(5.0014, grad_fn=<DivBackward1>),
 tensor(6.1255, grad_fn=<DivBackward1>),
 tensor(4.2725, grad_fn=<DivBackward1>),
 tensor(5.3381, grad_fn=<DivBackward1>),
 tensor(4.8708, grad_fn=<DivBackward1>),
 tensor(3.6831, grad_fn=<DivBackward1>),
 tensor(3.2103, grad_fn=<DivBackward1>),
 tensor(3.2966, grad_fn=<DivBackward1>),
 tensor(3.9925, grad_fn=<DivBackward1>),
 tensor(3.6426, grad_fn=<DivBackward1>),
 tensor(2.5486, grad_fn=<DivBackward1>),
 tensor(2.2693, grad_fn=<DivBackward1>),
 tensor(2.5763, grad_fn=<DivBackward1>),
 tensor(2.7698, grad_fn=<DivBackward1>),
 tensor(2.5545, grad_fn=<DivBackward1>),
 tensor(2.1910, grad_fn=<DivBackward1>),
 tensor(2.2293, grad_fn=<DivBackward1>),
 tensor(2.4382, grad_fn=<DivBackward1>),
 tensor(2.2021,